In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("train.csv")

In [10]:
df.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,target,Init_longitude
0,1372636858620000589,"[0, 0, 1]",NaN,NaN,1.138372,1372636858,"[1, 0, 0]",False,"[[-8.618643,41.141412],[-8.618499,41.141376],[...",330,[
1,1372637303620000596,"[0, 1, 0]",NaN,7.0,1.171510,1372637303,"[1, 0, 0]",False,"[[-8.639847,41.159826],[-8.640351,41.159871],[...",270,[
2,1372636951620000320,"[0, 0, 1]",NaN,NaN,-0.135061,1372636951,"[1, 0, 0]",False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-...",960,[
3,1372636854620000520,"[0, 0, 1]",NaN,NaN,0.811730,1372636854,"[1, 0, 0]",False,"[[-8.574678,41.151951],[-8.574705,41.151942],[...",630,[
4,1372637091620000337,"[0, 0, 1]",NaN,NaN,-0.054584,1372637091,"[1, 0, 0]",False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-...",420,[


In [3]:
df["target"] = df["POLYLINE"].apply(lambda x : max((x.count("[") - 2) *15, 0))

In [4]:
df = df.loc[df['MISSING_DATA'] == False]
# df = df.loc['target'] > 0


In [5]:
def one_hot(str):
    if str == 'A':
        return [1,0,0]
    elif str == 'B':
        return [0,1,0]
    elif str == 'C':
        return [0,0,1]
    else:
        return None

In [25]:
df["CALL_TYPE"]

0          C
1          B
2          C
3          C
4          C
          ..
1710665    C
1710666    C
1710667    C
1710668    B
1710669    B
Name: CALL_TYPE, Length: 1710660, dtype: object

In [6]:
df['CALL_TYPE'] = df['CALL_TYPE'].apply(lambda x: one_hot(x))
df['DAY_TYPE'] = df['DAY_TYPE'].apply(lambda x: one_hot(x))
df = df.loc[df['CALL_TYPE'] != None]
df = df.loc[df['DAY_TYPE'] != None]

In [ ]:
from datetime import datetime
def parse_time(x):
    dt = datetime.fromtimestamp(x["TIMESTAMP"])
    return dt.year, dt.month, dt.day, dt.hour, dt.weekday()

In [7]:
df['TAXI_ID'] = (df['TAXI_ID'] - df['TAXI_ID'].mean())/df['TAXI_ID'].std()

In [8]:
def normalize(col_name):
    df[col_name] = (df[col_name] - df[col_name].mean())/df[col_name].std()

In [11]:
type(df['POLYLINE'][0])

str

In [9]:
df['Init_longitude'] = df['POLYLINE'].apply(lambda x : x[0][0])
df['Init_latitude'] = df['POLYLINE'].apply(lambda x : x[0][1])
normalize('Init_longitude')
normalize('Init_latitude')

IndexError: string index out of range

In [ ]:
train_set = torch.Tensor(df.loc[:,['CALL_TYPE'],['TAXI_ID'], ['DAY_TYPE'], ['Init_longtitude'], ['Init_latitude'])